In [86]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [87]:
df=pd.read_csv('./titanic/train.csv')
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [88]:
df.shape

(891, 12)

### Lets clean our data again, same steps as the last time

We check for which values are empty. We can observe that the majority of the cabin values are empty so we will drop the column 'Cabin'. We drop the empty rows next, to clear out all the empty and missing values

In [89]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [90]:
df.drop(['Cabin'],axis=1,inplace=True)

In [91]:
df["Age"].fillna(df["Age"].median(skipna=True), inplace=True)
df["Fare"].fillna(df["Fare"].median(skipna=True), inplace=True)

In [92]:
df['TravelAlone']=np.where((df["SibSp"]+df["Parch"])>0, 0, 1)

In [93]:
training=pd.get_dummies(df, columns=["Pclass","Embarked","Sex"])
training.drop('Sex_female', axis=1, inplace=True)
training.drop('PassengerId', axis=1, inplace=True)
training.drop('Name', axis=1, inplace=True)
training.drop('Ticket', axis=1, inplace=True)

training['IsMinor']=np.where(training['Age']<=16, 1, 0)
training

Survived   Age  SibSp  Parch      Fare  TravelAlone  Pclass_1  Pclass_2  \
0           0  22.0      1      0    7.2500            0         0         0   
1           1  38.0      1      0   71.2833            0         1         0   
2           1  26.0      0      0    7.9250            1         0         0   
3           1  35.0      1      0   53.1000            0         1         0   
4           0  35.0      0      0    8.0500            1         0         0   
5           0  28.0      0      0    8.4583            1         0         0   
6           0  54.0      0      0   51.8625            1         1         0   
7           0   2.0      3      1   21.0750            0         0         0   
8           1  27.0      0      2   11.1333            0         0         0   
9           1  14.0      1      0   30.0708            0         0         1   
10          1   4.0      1      1   16.7000            0         0         0   
11          1  58.0      0      0   26.5500            1         1         0   
12          0  20.0      0      0    8.0500            1         0         0   
13          0  39.0      1      5   31.2750            0         0         0   
14          0  14.0      0      0    7.8542            1         0         0   
15          1  55.0      0      0   16.0000            1         0         1   
16          0   2.0      4      1   29.1250            0         0         0   
17          1  28.0      0      0   13.0000            1         0         1   
18          0  31.0      1      0   18.0000            0         0         0   
19          1  28.0      0      0    7.2250            1         0         0   
20          0  35.0      0      0   26.0000            1         0         1   
21          1  34.0      0      0   13.0000            1         0         1   
22          1  15.0      0      0    8.0292            1         0         0   
23          1  28.0      0      0   35.5000            1         1         0   
24          0   8.0      3      1   21.0750            0         0         0   
25          1  38.0      1      5   31.3875            0         0         0   
26          0  28.0      0      0    7.2250            1         0         0   
27          0  19.0      3      2  263.0000            0         1         0   
28          1  28.0      0      0    7.8792            1         0         0   
29          0  28.0      0      0    7.8958            1         0         0   
..        ...   ...    ...    ...       ...          ...       ...       ...   
861         0  21.0      1      0   11.5000            0         0         1   
862         1  48.0      0      0   25.9292            1         1         0   
863         0  28.0      8      2   69.5500            0         0         0   
864         0  24.0      0      0   13.0000            1         0         1   
865         1  42.0      0      0   13.0000            1         0         1   
866         1  27.0      1      0   13.8583            0         0         1   
867         0  31.0      0      0   50.4958            1         1         0   
868         0  28.0      0      0    9.5000            1         0         0   
869         1   4.0      1      1   11.1333            0         0         0   
870         0  26.0      0      0    7.8958            1         0         0   
871         1  47.0      1      1   52.5542            0         1         0   
872         0  33.0      0      0    5.0000            1         1         0   
873         0  47.0      0      0    9.0000            1         0         0   
874         1  28.0      1      0   24.0000            0         0         1   
875         1  15.0      0      0    7.2250            1         0         0   
876         0  20.0      0      0    9.8458            1         0         0   
877         0  19.0      0      0    7.8958            1         0         0   
878         0  28.0      0      0    7.8958            1         0         0   
879  

In [94]:
from sklearn.model_selection import train_test_split
labels=training['Survived']
train_data=training.drop(['Survived'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(train_data,labels,test_size=0.2)

In [95]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

In [96]:
model.fit(x_train,y_train)
prediction=model.predict(x_test)

In [97]:
model.score(x_test,y_test)

0.7821229050279329

### Feature Engineering time....
Let's make some new data now, and improve our poor little model

In [98]:
df['Title'] = df.Name.apply(lambda x: re.search(' ([A-Z][a-z]+)\.', x).group(1))

In [99]:
df['Fam_Size'] = df.Parch + df.SibSp

In [101]:
df.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Embarked  TravelAlone Title  Fam_Size  
0      0         A/5 21171   7.2500        S            0    Mr         1  
1      0          PC 17599  71.2833        C            0   Mrs         1  
2      0  STON/O2. 3101282   7.9250        S            1  Miss         0  
3      0            113803  53.1000        S            0   Mrs         1  
4      0            373450   8.0500        S            1    Mr         0

In [107]:
training=pd.get_dummies(df, columns=["Pclass","Embarked","Sex","Title"])
training.drop('PassengerId', axis=1, inplace=True)
training.drop('Name', axis=1, inplace=True)
training.drop('Ticket', axis=1, inplace=True)

training['IsMinor']=np.where(training['Age']<=16, 1, 0)
training.head()

Survived   Age  SibSp  Parch     Fare  TravelAlone  Fam_Size  Pclass_1  \
0         0  22.0      1      0   7.2500            0         1         0   
1         1  38.0      1      0  71.2833            0         1         1   
2         1  26.0      0      0   7.9250            1         0         0   
3         1  35.0      1      0  53.1000            0         1         1   
4         0  35.0      0      0   8.0500            1         0         0   

   Pclass_2  Pclass_3   ...     Title_Master  Title_Miss  Title_Mlle  \
0         0         1   ...                0           0           0   
1         0         0   ...                0           0           0   
2         0         1   ...                0           1           0   
3         0         0   ...                0           0           0   
4         0         1   ...                0           0           0   

   Title_Mme  Title_Mr  Title_Mrs  Title_Ms  Title_Rev  Title_Sir  IsMinor  
0          0         1          0         0          0          0        0  
1          0         0          1         0          0          0        0  
2          0         0          0         0          0          0        0  
3          0         0          1         0          0          0        0  
4          0         1          0         0          0          0        0  

[5 rows x 33 columns]

In [109]:
from sklearn.model_selection import train_test_split
labels=training['Survived']
train_data=training.drop(['Survived'],axis=1)
x_train,x_test,y_train,y_test=train_test_split(train_data,labels,test_size=0.2)

In [110]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

In [111]:
model.fit(x_train,y_train)
prediction=model.predict(x_test)

In [112]:
model.score(x_test,y_test)

0.8435754189944135

### Now we'll move onto some more powerful modesl

## THE MIGHTY SVM

In [113]:
from sklearn import svm

In [130]:
clf=svm.SVC(C = 30, gamma = 0.001)

In [131]:
clf.fit(x_train,y_train)

SVC(C=30, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [132]:
clf.score(x_test,y_test)

0.8212290502793296